# Rapport
- Balthazar Neveu
- balthazarneveu@gmail.com
- [Lab 2 on github](https://github.com/balthazarneveu/MVA23_SIGNAL/tree/lab_2)

Commençons par une première passe d'homogénéisation du jeu de donnée. En effet, l'échantillonnage temporal n'est pas régulier... l'horodatage n'est pas le même pour tous les signaux.
Afin de simplifier la visualisation et le travail sur les données, nous allons simplement ré-échantillonner le tout (timestamps réguliers, échantillonnage à 10HZ).
Chaque signal aura 100 échantillons ce qui permet d'avoir un volume raisonnable et même de supperposer les différents signaux pour les comparer.

In [ ]:
from utilities import homogenize_dataset, get_hand_crafted_features, ALL_CLASSIFIERS, train_classifier, get_better_features, extract_peaks
from matplotlib import pyplot as plt
import scipy

import numpy as np
%load_ext autoreload
%autoreload 2

In [ ]:
from utilities import homogenize_dataset
num_samples = 100
df = df_train = homogenize_dataset(dataset="train", num_samples=num_samples)
df_test = homogenize_dataset(dataset="test", num_samples=num_samples)
extract_peaks(df_train, add_to_df=True)
extract_peaks(df_test, add_to_df=True)
df.head()


In [ ]:
x_train, y_train, labels_features = get_better_features(df_train)
x_test, y_test, _ = get_better_features(df_test)

In [ ]:
# whiten=True
whiten=False
x_train, y_train, labels_features = get_better_features(df_train, whiten=whiten)
x_test, y_test, _ = get_better_features(df_test,  whiten=whiten)

from utilities import ALL_CLASSIFIERS, DECISION_TREE, RANDOM_FOREST, SVM, ADABOOST, XG_BOOST
COLOR_LIST = "rgbckyp"
best_accuracies_overall = []
best_feature_dimensions = []
# classifiers_list = ALL_CLASSIFIERS #[DECISION_TREE, RANDOM_FOREST, SVM, ADABOOST]
# classifiers_list = [SVM]
classifiers_list = [XG_BOOST,  DECISION_TREE, RANDOM_FOREST, ADABOOST, SVM]
scanned_feature_dimension = list(range(1, len(labels_features)+1))
plt.figure(figsize=(10, 10))
for classifier_index, classifier_type in enumerate(classifiers_list):
    best_accuracies = []
    feature_dimensions = []
    color = COLOR_LIST[classifier_index%len(COLOR_LIST)]
    for feature_dimension in scanned_feature_dimension:
        accuracies, best_depth, _ = train_classifier(x_train, x_test, y_train, y_test, feature_dimension=feature_dimension, debug=False,  show=False, classifier=classifier_type)
        # print(f"#features={feature_dimension} Tree depth={best_depth} accuracy training {accuracies[0]*100:.1f}% | accuracy test {accuracies[1]*100:.1f}%")
        best_accuracies.append(accuracies)
        feature_dimensions.append(feature_dimension)
    # plt.plot(feature_dimensions, 100.*np.array(best_accuracies)[:, 0], color+"--", alpha=0.1) #label=f"{classifier_type} accuracy training")
    plt.plot(feature_dimensions, 100.*np.array(best_accuracies)[:, 1], color+"-") #label=f"{classifier_type} accuracy validation")
    best_index = np.argmax(np.array(best_accuracies)[:, 1])
    best_accuracy = best_accuracies[best_index][1]
    best_feature_dimension = feature_dimensions[best_index]
    plt.plot(feature_dimensions[best_index], 100.*best_accuracy, color+"o", label=f"{classifier_type} Accuracy {100*best_accuracy:.1f}% - {best_feature_dimension} features")
    plt.legend()
    best_accuracies_overall.append(best_accuracy)
    best_feature_dimensions.append(best_feature_dimension)
best_classifier_index = np.argmax(np.array(best_accuracies_overall))
plt.title(f"Best accuracy {100*best_accuracies_overall[best_classifier_index]:.1f} %\n"+ 
          f" obtained with {classifiers_list[best_classifier_index]}" +
          f" on {best_feature_dimensions[best_classifier_index]} features")
new_labels = labels_features[:len(scanned_feature_dimension)]
new_labels = [f"{label}\n{feature}" for label, feature in zip(new_labels, scanned_feature_dimension)]
plt.xticks(scanned_feature_dimension, new_labels, rotation=80)
plt.xlabel("Number of features")
plt.ylabel(r"Accuracy (%)")
plt.ylim(70, 100)
plt.grid()
plt.show()



## Phase exploratoire
On commence par chercher des critères simples (statistiques basiques des signaux)  pour classifier les données.

### Min puissance
- Si on fait défiler les signaux de puissance en figeant l'échelle verticale, nous remarquons clairement un "paquet" au dessus des autres.
- Un descripteur très simple pour discriminer la menace correspond au mimimum de la puissance. On sait déjà qu'il n'est pas suffisant (au vu du gros "paquet" vert et rouge)

![](figures/superposition_signaux_puissance.png)


### Moyenne et puissance des fréquences

- L'étude de l'évolution temporelle des fréquences ne révèle pas de fluctuations perceptibles (signaux stationnaires).

![](figures/superposition_signaux_frequence.png)

- On peut simplement évaluer les moments et ainsi tracer la distribution moyenne des fréquences ainsi que l'écart type



En poussant un peu l'étude manuelle des caractéristiques, on commence à arriver à distinguer quelques groupes:
- écart type(1/fréquence)/moyenne(1/fréquence)
- min(puissance)

![](figures/x=freq_avg_y=freq_std_div_freq_avg.png)


Un arbre de décision devrait permettre de classifier les données (partitionner l'espace en plusieurs rectangles).


## Classification basique
Une approche rationnelle consiste à aggréger les caractéristiques et entraîner un classifieur simple (arbre de décisions) en jouant sur sa profondeur.
Plus on ajoute de caractéristiques, plus on espère que la précision du modèle augmente


![decision_tree](figures/decision_tree.png)

- Avec une unique caractéristique (bien choisie) et un unique seuil, nous atteignons 70% de précision.
- En augmentant la profondeur de l'arbre de décision jusqu'à 4, nous atteignons une précision de 75%
- Avec 2 caractéristiques, nous remarquons qu'il est nécessaire d'avoir une profondeur de 4 sur l'arbre de décision afin d'atteindre la précision optimale.
- Nous remarquons qu'au delà d'une profondeur de 6, l'arbre de décision est à la limite de "sur-apprentissage" (la précision sur la base d'entraînement augmente alors que la précision de validation stagne ou commence à diminuer.)

### Basic classifier

In [ ]:
# Define the training and testing sets by extracting the features from the dataframes.
x_train, y_train = get_hand_crafted_features(df_train)
x_test, y_test = get_hand_crafted_features(df_test)

In [ ]:
compare_plots = True
plt.figure(figsize=(10, 10))
for feature_dimension in range(1, 5):
    accuracies, best_depth, _ = train_classifier(x_train, x_test, y_train, y_test, feature_dimension=feature_dimension, debug=True,  show=not compare_plots)
    print(f"#features={feature_dimension} Tree depth={best_depth} accuracy training {accuracies[0]*100:.1f}% | accuracy test {accuracies[1]*100:.1f}%")
if compare_plots:
    plt.grid()
    plt.show()

# Analyzing the time series

Let's first retieve some remaining "difficult" signals to classify.

In [ ]:
features_to_keep = 2
accuracies, best_depth, classifier = train_classifier(x_train, x_test, y_train, y_test, feature_dimension=features_to_keep, debug=False,  show=False, forced_depth=4)
y_train_pred = classifier.predict(x_train[:, :features_to_keep])
missclassified_indexes = np.where(y_train_pred!=y_train)[0]

In [ ]:
all_indexes = df.index.values
# Get indexes that are not in missclassified_indexes
not_missclassified_indexes = np.setdiff1d(all_indexes, missclassified_indexes)

# Generate
num_indexes_to_choose = 100  # specify the number of indexes you want
random_indexes = np.random.choice(not_missclassified_indexes, num_indexes_to_choose, replace=False)
df_random = df.take(random_indexes).copy().reset_index()

In [ ]:
df_hard = df.take(missclassified_indexes).copy()
df_hard.reset_index(inplace=True)

In [ ]:
df_hard.menace.sum()/len(df_hard)

In [ ]:
plt.plot(np.array([df_hard.puissance[idx] for idx in range(len(df_hard)) if df_hard.menace[idx]]).T, "r", alpha=0.05, label="menace")
plt.plot(np.array([df_hard.puissance[idx] for idx in range(len(df_hard)) if not df_hard.menace[idx]]).T, "g", alpha=0.05, label="safe")
plt.plot(np.array([df_random.puissance[idx] for idx in range(len(df_random)) if not df_random.menace[idx]]).T, "g", alpha=0.1, label="safe")
plt.grid()
plt.show()

In [ ]:
peaks_min, _props =  scipy.signal.find_peaks(-df_hard.puissance[0])
peaks, props =  scipy.signal.find_peaks(df_hard.puissance[0])
plt.plot(df_hard.puissance[0], "k-", label="power")
plt.plot(peaks, df_hard.puissance[0][peaks], "xr", label="extracted peaks")
plt.plot(peaks_min, df_hard.puissance[0][peaks_min], "xb", label="min extracted peaks")
plt.grid()
plt.title("Extraction of peaks from the power signal")
plt.legend()
plt.show()
# np.average(peaks[1:] - peaks[:-1])

In [ ]:
peaks, peak_vals = extract_peaks(df, add_to_df=True)
plt.hist([len(el) for idx, el in enumerate(df.peaks_loc) if df.menace[idx]], color="r", bins=40, label="menace", density=True)
plt.hist([len(el) for idx, el in enumerate(df.peaks_loc) if not df.menace[idx]], color="g", bins=40, alpha=0.5, label="safe", density=True)
plt.title("Histogram of number of peaks per signal")
plt.grid()
plt.legend()
plt.show()

In [ ]:
plt.hist([el for idx, el in enumerate(df.impulsion_freq) if df.menace[idx]], color="r", bins=100, label="menace", density=True)
plt.hist([el for idx, el in enumerate(df.impulsion_freq) if not df.menace[idx]], color="g", bins=100, alpha=0.5, label="safe", density=True)
plt.title("Histogram of impulsion periods")
plt.grid()
plt.xlim(0, 7500)
plt.legend()
plt.show()

In [ ]:
colors = ["r" if el else "g" for el in df["menace"]]
fig = plt.figure()
ax = fig.add_subplot(projection='3d')
ax.scatter(x_train[:, 2], x_train[:, 0], x_train[:, 1], color=colors)
plt.show()
